Some chatgpt junk i got. But has some ideas on how to set prior given the noise level relative to signal level from the pretest experiments

Basically it allows for uncertainty quantification across 2 axis while setting the others at 0.5 (between (0,1)), which could be indicative of component interactions and relations when disentagling models components to understand their influence on output.

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from botorch.models import SingleTaskGP
from botorch.acquisition import qNoisyExpectedImprovement
from botorch.optim import optimize_acqf
from botorch.utils.sampling import draw_sobol_samples
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.likelihoods import GaussianLikelihood
from botorch.fit import fit_gpytorch_mll

# -----------------------------
# SETTINGS
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

d = 4  # media components
bounds = torch.tensor([[0.0]*d, [1.0]*d], device=device, dtype=dtype)

sigma_hat2 = 0.03**2   # residual variance from pilot
n_initial_candidates = 12
n_reps_explore = 3
n_candidates_per_round = 32
n_confirm = 5
top_k_confirm = 5
n_rounds = 3  # number of BO rounds
mde_threshold = 0.05  # minimum detectable effect

# -----------------------------
# INITIAL DOE
# -----------------------------
X_init = draw_sobol_samples(bounds=bounds, n=1, q=n_initial_candidates).squeeze(0)
X_init = X_init.repeat_interleave(n_reps_explore, dim=0)
Y_init = torch.rand(X_init.shape[0], 1, device=device, dtype=dtype)  # placeholder

# -----------------------------
# GP WITH FIXED NOISE
# -----------------------------
likelihood = GaussianLikelihood(noise=torch.tensor(sigma_hat2), noise_constraint=None)
gp = SingleTaskGP(X_init, Y_init, likelihood=likelihood)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

# -----------------------------
# HELPER FUNCTION: One BO round
# -----------------------------
def run_bo_round(gp, n_candidates, n_reps, mde_threshold=None):
    # Acquisition function
    qnei = qNoisyExpectedImprovement(model=gp, X_baseline=gp.train_inputs[0])
    
    # Optimize acquisition
    candidates, _ = optimize_acqf(
        acq_function=qnei,
        bounds=bounds,
        q=n_candidates,
        num_restarts=10,
        raw_samples=100,
    )
    
    # Repeat for replicates
    X_new = candidates.repeat_interleave(n_reps, dim=0)
    
    # Placeholder: replace with actual experimental measurements
    Y_new = torch.rand(X_new.shape[0], 1, device=device, dtype=dtype)
    
    # Ensure Y_new is 2D
    if Y_new.ndim == 1:
        Y_new = Y_new.unsqueeze(-1)
    
    # Update GP
    X_train = torch.cat([gp.train_inputs[0], X_new], dim=0)
    Y_train = torch.cat([gp.train_targets, Y_new], dim=0)
    gp.set_train_data(X_train, Y_train, strict=False)
    fit_gpytorch_mll(ExactMarginalLogLikelihood(gp.likelihood, gp))
    
    # Compute summary statistics
    with torch.no_grad():
        posterior = gp.posterior(candidates)
        mean_pred = posterior.mean.squeeze()
        std_pred = posterior.variance.sqrt().squeeze()
        snr = mean_pred / std_pred
        top2_gap = mean_pred.max() - mean_pred.topk(2).values.min()
    
    summary = pd.DataFrame({
        'mean_pred': mean_pred.cpu().numpy(),
        'std_pred': std_pred.cpu().numpy(),
        'snr': snr.cpu().numpy(),
        'top2_gap': float(top2_gap.cpu().numpy())
    })
    
    stop_flag = False
    if mde_threshold is not None and top2_gap < mde_threshold:
        stop_flag = True
    
    return gp, candidates, Y_new, summary, stop_flag

# -----------------------------
# MAIN BO LOOP
# -----------------------------
all_summary = []

for round_idx in range(n_rounds):
    print(f"\n=== BO ROUND {round_idx+1} ===")
    
    # Exploration phase
    gp, new_candidates, Y_new, summary, stop_flag = run_bo_round(
        gp, n_candidates_per_round, n_reps_explore, mde_threshold
    )
    all_summary.append(summary)
    
    print("Top 5 predicted candidates this round:")
    print(summary.sort_values('mean_pred', ascending=False).head(5))
    
    if stop_flag:
        print(f"Stopping BO: top2 gap {summary['top2_gap'].iloc[0]:.3f} < MDE {mde_threshold}")
        break
    
    # Confirmation phase
    top_idx = summary['mean_pred'].nlargest(top_k_confirm).index
    X_top = new_candidates[top_idx].repeat_interleave(n_confirm, dim=0)
    
    # Placeholder: replace with actual measurements
    Y_top = torch.rand(X_top.shape[0], 1, device=device, dtype=dtype)
    
    # Ensure Y_top is 2D
    if Y_top.ndim == 1:
        Y_top = Y_top.unsqueeze(-1)
    
    X_train = torch.cat([gp.train_inputs[0], X_top], dim=0)
    Y_train = torch.cat([gp.train_targets, Y_top], dim=0)
    gp.set_train_data(X_train, Y_train, strict=False)
    fit_gpytorch_mll(ExactMarginalLogLikelihood(gp.likelihood, gp))
    
    # 2D slice plotting (components 1 vs 2)
    n_grid = 20
    x1 = torch.linspace(0, 1, n_grid)
    x2 = torch.linspace(0, 1, n_grid)
    X1, X2 = torch.meshgrid(x1, x2, indexing='ij')
    X_grid = torch.stack([X1.flatten(), X2.flatten(),
                          torch.full_like(X1.flatten(), 0.5),
                          torch.full_like(X1.flatten(), 0.5)], dim=-1).to(device).type(dtype)
    
    with torch.no_grad():
        posterior_grid = gp.posterior(X_grid)
        mean_grid = posterior_grid.mean.cpu().numpy().reshape(n_grid, n_grid)
        std_grid = posterior_grid.variance.sqrt().cpu().numpy().reshape(n_grid, n_grid)
    
    plt.figure(figsize=(8,6))
    plt.contourf(X1.cpu(), X2.cpu(), mean_grid, levels=20, cmap='viridis')
    plt.colorbar(label='Predicted Growth Rate')
    plt.contour(X1.cpu(), X2.cpu(), std_grid, levels=5, colors='white', linestyles='--')
    plt.xlabel("Media component 1 (normalized)")
    plt.ylabel("Media component 2 (normalized)")
    plt.title(f"BO Round {round_idx+1}: Mean & Uncertainty")
    plt.show()

# -----------------------------
# EXPORT SUMMARY
# -----------------------------
summary_all_rounds = pd.concat(all_summary, keys=[f"Round_{i+1}" for i in range(len(all_summary))])
summary_all_rounds.to_csv("bo_summary.csv")
print("\nSummary exported to bo_summary.csv")



=== BO ROUND 1 ===


c:\Users\uqkmuroi\gitcode\Yarrowia_media_optimizer\venv\Lib\site-packages\botorch\acquisition\monte_carlo.py:502: NumericsWarning: qNoisyExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qNoisyExpectedImprovement 	 --> 	 qLogNoisyExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)
c:\Users\uqkmuroi\gitcode\Yarrowia_media_optimizer\venv\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


RuntimeError: Tensors must have same number of dimensions: got 1 and 2